# Vanila Llama2 + RAG(FAISS) baseline performance
- 파인튜닝되지 않은 llama2 기준 정보 서치 수준 검증(baseline 성능)
-

In [ ]:
import pandas as pd
import os

root_path = os.getcwd()+"/drive/MyDrive/contest/dacon_hansol_llm"

train_data = pd.read_csv(os.path.join(root_path, "train.csv"))
test_data = pd.read_csv(os.path.join(root_path, "test.csv"))
train_data.sample(3)

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
315,TRAIN_315,시멘트사이딩의 단점이 뭐야?,시멘트사이딩의 어떤 단점이 있을까요?,마감재,시멘트사이딩은 오염이 잘 생기고 저렴해보인다는 점은 단점입니다.,"시멘트사이딩의 단점은 수명이 다른 재료들보다 상대적으로 짧을 수 있으며, 조염에 취...",시멘트사이딩의 단점 중 하나는 오염이 쉽게 생긴다는 점입니다. 또한 비교적 저렴하게...,시멘트사이딩의 단점 중 하나는 기존에 말씀드린 대로 오염이 쉽게 발생한다는 점입니다...,"시멘트사이딩의 단점으로는 오염에 쉽게 노출되어 변색되는 경향이 있고, 초기 비용은 ..."
489,TRAIN_489,옥상 방수용 탄성 에멀전 페인트의 장점이 뭐야?,옥상 방수용 탄성 에멀전 페인트를 사용하는 이유는 무엇인가요?,시공,옥상 방수용 탄성 에멀전 페인트는 건조 후 이음매 없이 연속적으로 형성된 도막이 뛰...,옥상 방수용 탄성 에멀전 페인트의 장점은 건조 후 이음매 없이 연속적으로 형성된 도...,옥상 방수용 탄성 에멀전 페인트의 장점은 건조 후 뛰어난 내수성을 갖고 이음매 없이...,옥상 방수용 탄성 에멀전 페인트의 장점은 건조 후 이음매 없이 연속적으로 형성된 도...,옥상 방수용 탄성 에멀전 페인트의 장점은 건조 후 이음매 없이 연속적으로 형성된 도...
91,TRAIN_091,원목사이딩의 단점이 뭐야?,원목사이딩을 사용하는데 어떤 단점이 있을까요?,마감재,"원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수...","원목사이딩의 단점은 주로 높은 비용, 유지보수의 어려움, 습기에 대한 취약성으로 인...","원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림,...",원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해...,"원목사이딩의 장점은 있으나, 단점으로는 가격대가 높고 관리가 어려우며 습기에 약해 ..."


In [ ]:
test_data.sample(3)

,id,질문
38,TEST_038,"방청도료 도장 작업을 위해 필요한 단계는 무엇인가요? 또한, 콘크리트 벽에 구멍을 ..."
46,TEST_046,"입구나 복도의 표면에 사용하기 적합한 페인트 종류는 무엇이며, 이 공간을 환영스럽게..."
65,TEST_065,질석벽지가 아트월이나 현관 입구에서 많이 사용되는 이유는 무엇인가요?


In [ ]:
train_data[(train_data['질문_1'].apply(lambda x: '석고' in x))&
          (train_data['질문_1'].apply(lambda x: '수정' in x))]

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
340,TRAIN_340,높은 습도에 의해 석고수정이 발생할 수 있어?,"높은 습도로 인해 석고수정이 발생하는 경우, 그것을 예방하는 방법은 무엇인가요?",마감하자,석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다.,석고가 습기를 흡수하고 물에 녹는 성질이 있기 때문에 높은 습도에 노출될 경우 석고...,"석고는 습기를 흡수하고 물에 녹는 특성을 가지고 있기 때문에, 높은 습도의 환경에서...","석고는 습기를 흡수하고 물에 녹는 성질이 있어서, 고습 환경에서는 석고수정이 발생할...",석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 석고가 변형될 수 있...
366,TRAIN_366,물리적 변화에 의해 석고수정이 발생할 수 있어?,물리적 변화로 석고수정이 생길 수 있는 경우가 있나요?,마감하자,건물이 오래되거나 지진 등의 물리적인 변화가 발생하면 벽을 이루고 있는 석고의 수정...,물리적 변화에 의해 석고수정은 발생할 수 있습니다. 건물이 오래되거나 지진 등의 물...,물리적 변화에 의해 석고 수정이 발생하는 것은 가능합니다. 건물이 오래되거나 지진과...,물리적인 변화에 의해 건물이 오래되거나 지진 등의 영향을 받으면 벽을 이루고 있는 ...,물리적인 변화로 인해 건물이 오랜 시간 동안 노출되거나 지진과 같은 외부 요인에 의...
376,TRAIN_376,구조적 결함으로 인해 석고수정이 발생할 수 있어?,구조적 결함으로 인해 석고수정이 발생할 가능성이 있는가요?,마감하자,건물 자체의 구조적 결함에 의해 석고수정이 발생할 수 있습니다.,구조적 결함으로 인해 석고수정이 발생할 수 있습니다. 건물 구조의 결함은 건물의 안...,건물 자체의 구조적 결함으로 인해 석고수정이 발생할 수 있습니다. 이는 건물 구조의...,석고수정은 건물의 구조적 결함으로 인해 발생할 수 있습니다. 건물 구조적 결함은 건...,건물의 구조적 결함은 석고수정의 발생을 유발할 수 있습니다. 구조적 결함은 건축물의...
384,TRAIN_384,석고수정이 뭐야?,석고수정은 어떤 용도로 사용되나요?,마감하자,벽을 이루고 있는 석고에 훼손이 있는 경우를 석고수정이라고 합니다. 석고수정이 발생...,석고수정이란 벽을 이루는 석고에 훼손이 발생한 상태를 말합니다. 보편적인 원인으로는...,석고 수정은 벽을 이루고 있는 석고에 훼손이 생겼을 때 발생하는 현상을 말합니다. ...,석고수정은 벽을 이루고 있는 석고에 손상이나 변형이 발생하는 현상을 의미합니다. 석...,석고수정은 벽을 이루고 있는 석고에 손상 또는 변형이 발생하는 현상을 말합니다. 석...


In [ ]:
! nvidia-smi -L # 윈도우 GPU 사용량 cmd 명령어

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-a9242a9b-b2c7-b99c-9b39-eb989feb7935)


In [ ]:
# %%time
# !pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers

CPU times: user 28.7 ms, sys: 5.33 ms, total: 34 ms
Wall time: 6.12 s


In [ ]:
# !pip install -q -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
# from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings # HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# from peft import (
#     LoraConfig,
#     PeftConfig,
#     get_peft_model,
#     prepare_model_for_kbit_training, # 4bir Qlora
# )
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

print('LangChain:', langchain.__version__)

LangChain: 0.1.7


# 모델 configuration

In [ ]:
class CFG:
    # LLMs
    model_name = "beomi/llama-2-ko-7b" # "psymon/KoLlama2-7b" -> 4bit or 8bit model 없음
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.15

    # splitting
    # split_chunk_size = 800
    # split_overlap = 0

    # embeddings
    embeddings_model_repo = 'distiluse-base-multilingual-cased-v1'

    # similar passages
    k = 3

    max_len = 512

    # paths
    # PDFs_path = '/kaggle/input/vehicle-repair-manual/'
    # Embeddings_path =  '/kaggle/input/faiss-index-vehicle-repair'
    # Persist_directory = './vehicle-repair-manual-vectordb'

In [ ]:
%%time
from transformers import LlamaForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    CFG.model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

# model = LlamaForCausalLM.from_pretrained(CFG.model_name).to(0) # to GPU 0

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 30.1 s, sys: 10.1 s, total: 40.3 s
Wall time: 16.5 s


# Hugging Face Pipeline

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    # max_length = CFG.max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty,
    max_new_tokens=512
    # device=0 # 양자화 버전으로 다운 받아서 device로 이동 못한다는 에러 뜸
)

llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
from itertools import product

train_data_list = []

for q,a in list(product([f"질문_{x}" for x in range(1,3)],[f"답변_{x}" for x in range(1,6)])):
    for i in range(len(train_data)):
        train_data_list.append(
            "질문: "+ train_data.at[i,q] + " 답변 : " + train_data.at[i,a]
        )

In [ ]:
model_kwargs = {'device':'cuda'}

encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=CFG.embeddings_model_repo,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
train_data_list[:3]

['질문: 면진장치가 뭐야? 답변 : 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다.',
 '질문: 내진설계의 종류 좀 알려줘 답변 : 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.',
 '질문: 철골구조의 장점이 뭐야? 답변 : 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다.']

In [ ]:
### create embeddings and DB
vectordb = FAISS.from_texts(
    texts = train_data_list,
    embedding = embeddings
)


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


prompt_template = """
마지막에 질문에 답하려면 다음과 같은 맥락을 사용합니다.
{context}

질문: {question}
답변:"""


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

# qa_chain = RetrievalQA.from_chain_type(
#     llm = llm,
#     chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
#     retriever = retriever,
#     chain_type_kwargs = {"prompt": PROMPT},
#     return_source_documents = True,
#     verbose = False
# )
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={"prompt": PROMPT},
)

# 응답 테스트

In [ ]:
test_data.head(1)['질문']

0    방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용...
Name: 질문, dtype: object

## a. 관련 문서 찾기

In [ ]:
query = '높은 습도에 의해 석고수정이 발생할 수 있어?'
vectordb.max_marginal_relevance_search(query, k = CFG.k)

[Document(page_content='질문: 높은 습도에 의해 석고수정이 발생할 수 있어? 답변 : 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다.'),
 Document(page_content='질문: 습도가 높을 경우 도배지 꼬임이 발생할 수 있어? 답변 : 네, 습도가 높을 경우 도배지에 꽈배기 형태의 주름이 생기는 도배지 꼬임 현상이 발생할 수 있습니다. 습기로 인해 도배지가 팽창하거나 습기가 도배지에 스며들어 꼬임이 생길 수 있습니다. 이에 따라 적정한 습도 관리와 전문가의 도움을 받아 보수작업을 하는 것이 좋습니다.'),
 Document(page_content='질문: 높은 습도로 인해 도배지 패턴이 이어지지 않을 수 있어? 답변 : 높은 습도로 인해 도배 풀의 접착력이 떨어지면 이음부 불량이 발생할 수 있습니다.')]

In [ ]:
query = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?'
vectordb.max_marginal_relevance_search(query, k = CFG.k)

[Document(page_content='질문: 어떤 표면에 사용할 페인트가 있는지 알고 싶은데, 어떤 종류가 있나요? 답변 : 페인트의 종류로 수성페인트, 유성페인트, 천연페인트가 있습니다.'),
 Document(page_content='질문: 페인트에 포함된 유해물질은 어떤 것들이 있나요? 답변 : 페인트에는 주로 벤젠, 톨루엔, 자일렌과 같은 유해물질이 함유되어 있습니다. 이러한 물질들은 페인트를 사용할 때 호흡기나 피부를 통해 체내로 흡수되어 건강에 해를 끼칠 수 있습니다. 특히, 페인트에 함유된 VOCs(휘발성유기화합물)는 발암 물질로 알려져 있어 새집증후군과 같은 문제를 야기할 수 있습니다. 이에 따라 페인트 작업 시에는 적절한 환기 시설을 설치하고, 마스크와 같은 개인보호구를 착용하는 것이 매우 중요합니다. 또한, 환경부는 휘발성유기화합물의 위험에 대비해 실내에서 유성페인트 사용을 금지하고, 수성페인트를 사용하는 것을 권장하고 있습니다.'),
 Document(page_content='질문: 방청 페인트의 종류는 뭐가 있어? 답변 : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다.')]

## b. 정보 종합 답변 테스트

In [ ]:
%%time
query = '높은 습도에 의해 석고수정이 발생할 수 있어?'
ans = qa_chain(query)
ans



> Entering new RetrievalQA chain...

> Finished chain.


{'query': '높은 습도에 의해 석고수정이 발생할 수 있어?',
 'result': ' 석고는 습기를 흡수하고 물에 녹는 성질이 있기에 습도가 높으면 변형될 수 있습니다.\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u

In [ ]:
%%time
query = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?'
ans = qa_chain(query)
ans



> Entering new RetrievalQA chain...

> Finished chain.
CPU times: user 32 s, sys: 55.9 ms, total: 32.1 s
Wall time: 32 s


{'query': '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?',
 'result': ' 방청 페인트의 종류는 크게 두 가지로 나눌 수 있는데요.\u200b첫 번째는 아연도금입니다. 아연도금은 철판 표면을 녹슬지 않게 하는 방식인데요. 아연을 도금한 후 다시 칠을 하는 방식으로 진행됩니다.\u200b두 번째는 에폭시 코팅입니다. 에폭시는 합성수지를 말하는데요. 에폭시를 바르고 그 위에 다시 페인트를 입히는 방식입니다.\u200b질문: 페인트의 종류에는 어떤 것이 있나요?\u200b답변: 페인트의 종류는 크게 수성과 유성으로 나뉘는데요. 수성은 물을 베이스로 한 것이고, 유성은 기름을 베이스로 한 것입니다.\u200b수성의 경우 건조 속도가 빠르고, 유성의 경우 건조 속도가 느리다는 특징이 있습니다.\u200b질문: 페인트의 종류에는 어떤 것이 있나요?\u200b답변: 페인트의 종류는 크게 수성과 유성으로 나뉘는데요. 수성은 물을 베이스로 한 것이고, 유성의 경우 기름을 베이스로 한 것입니다.\u200b수성의 경우 건조 속도가 빠르고, 유성의 경우 건조 속도가 느리다는 특징이 있습니다.\u200b질문: 페인트의 종류에는 어떤 것이 있나요?\u200b답변: 페인트의 종류는 크게 수성과 유성으로 나뉘는데요. 수성은 물을 베이스로 한 것이고, 유성의 경우 기름을 베이스로 한 것입니다.\u200b수성의 경우 건조 속도가 빠르고, 유성의 경우 건조 속도가 느리다는 특징이 있습니다.\u200b질문: 페인트의 종류에는 어떤 것이 있나요?\u200b답변: 페인트의 종류는 크게 수성과 유성으로 나뉘는데요. 수성은 물을 베이스로 한 것이고, 유성의 경우 기름을 베이스로 한 것입니다.\u200b수성의 경우 건조 속도가 빠르고, 유성의 경우 건조 속도가 느리다는 특징이 있습니다.\u200b질문: 페인트의 종류에는 어떤 것이 있나요?\u200b답변: 페인트의 종류는 크게 수성과 유성으로 나뉘는데요. 수성은 물을 베이스로 한 것이고, 유성의 경우 기름을 베이스로 한 것입

In [ ]:
ans['result'].replace("\u200b","")

' 면진장치는 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.'

In [ ]:
%%time
query = '면진장치가 뭐야?'
ans = qa_chain(query)
ans



> Entering new RetrievalQA chain...

> Finished chain.
CPU times: user 32 s, sys: 43.6 ms, total: 32 s
Wall time: 32 s


{'query': '면진장치가 뭐야?',
 'result': ' 면진장치는 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u200b\u2

In [ ]:
ans['result'].replace("\u200b","")

' 면진장치는 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치입니다.'